In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, PySimpleGUI as sg, ipywidgets as widgets
import os, re, requests
from bs4 import BeautifulSoup
from IPython.display import display, HTML, Markdown

## GUI tool beginnings

In [2]:
# # Initialize global variables
# names = []

# def load_dataframes(folder_path):
#     global names
#     all_files = os.listdir(folder_path)
#     pkl_files = [file for file in all_files if file.endswith('.pkl')]
#     file_path_list = [os.path.join(folder_path, file) for file in pkl_files]

#     names = [name.strip('.pkl') for name in pkl_files]
#     dflist = list(zip(names, file_path_list))

#     for name, df in dflist:
#         exec(f"df_{name} = pd.read_pickle(df)")
#         print(f"{df} loaded as df_{name}")

In [3]:
# names = []
# # Define the layout of the GUI
# layout = [
#     [sg.Text('Select Folder for PKL Files')],
#     [sg.InputText(key='-FOLDER-'), sg.FolderBrowse(target='-FOLDER-')],
#     [sg.Button('Load Data')],
#     [sg.Text('Select DataFrame to View:'), sg.InputCombo(values=[], key='-DF_SELECT-', size=(20, 1))],
#     [sg.Button('View Head')],
#     [sg.Output(size=(60, 18))]
# ]

# # Create the window
# window = sg.Window('Read Data', layout)

# while True:
#     event, values = window.read()

#     if event == sg.WIN_CLOSED:
#         break

#     if event == 'Load Data':
#         folder_path = values['-FOLDER-']
#         load_dataframes(folder_path)
        
#         # Update the dropdown with available DataFrame names
#         window['-DF_SELECT-'].update(values=names)
    
#     if event == 'View Head':
#         selected_df_name = values['-DF_SELECT-']
#         if selected_df_name:
#             try:
#                 selected_df = globals()[f'df_{selected_df_name}']  # Add 'df_' prefix
#                 print(f"Head of df_{selected_df_name}:\n{selected_df.head()}")
#             except KeyError:
#                 print(f"DataFrame df_{selected_df_name} not found.")

# window.close()

## Manual Cleaning

In [4]:
# Get a list of all files in the folder
all_files = os.listdir('Vision/')
# Filter the list to include only CSV files
pkl_files = [file for file in all_files if file.endswith('.pkl')]
# make a list of full paths
file_path_list = ['Vision/' + file for file in pkl_files]

# pretty sure this wasn't serving any purpose

# # Extract the filenames and full paths
# full_paths = [os.path.abspath(os.path.join('Vision/', file)) for file in pkl_files]

In [5]:
names = [name.strip('.pkl') for name in pkl_files]

# Pair names with file paths using zip
dflist = list(zip(names, file_path_list))

# Create dataframes from all with matching names
for name, df in dflist:
    exec(f"df_{name} = pd.read_pickle(df)")

dflist

[('AR', 'Vision/AR.pkl'),
 ('CL', 'Vision/CL.pkl'),
 ('EMProjectAssoc', 'Vision/EMProjectAssoc.pkl'),
 ('LD', 'Vision/LD.pkl'),
 ('LedgerAR', 'Vision/LedgerAR.pkl'),
 ('Opportunity', 'Vision/Opportunity.pkl'),
 ('OpportunityCustomTabFields', 'Vision/OpportunityCustomTabFields.pkl'),
 ('PR', 'Vision/PR.pkl'),
 ('Projects_BillingProjections', 'Vision/Projects_BillingProjections.pkl'),
 ('Projects_ProjectSetu', 'Vision/Projects_ProjectSetup.pkl')]

In [6]:
null_counts = []
for column in df_AR.columns:
    null_counts.append((column, str((df_AR[column].notnull().sum() / len(df_AR[column]) * 100)) + "% not null"))
    
# null_counts will be a list of tuples, where each tuple contains the column name and the percentage of non-null values.
null_counts

[('WBS1', '100.0% not null'),
 ('WBS2', '100.0% not null'),
 ('WBS3', '100.0% not null'),
 ('Invoice', '100.0% not null'),
 ('Period', '100.0% not null'),
 ('InvoiceDate', '100.0% not null'),
 ('PaidPeriod', '100.0% not null'),
 ('LinkCompany', '0.0% not null'),
 ('InvBalanceSourceCurrency', '100.0% not null'),
 ('DueDate', '0.0% not null'),
 ('RetainageDate', '1.4384006934662477% not null')]

In [7]:
# DueDate & LinkCompany are null, and RetainageDate only has 531 values, & is unlikely to be useful
df_AR.drop("DueDate", axis=1, inplace=True)
df_AR.drop("LinkCompany", axis=1, inplace=True)
df_AR.drop("RetainageDate", axis=1, inplace=True)

In [8]:
def dataframe_info(selected_df):
    
    # Print shape
    print(f"Shape: {selected_df.shape}\n")
    
    # Print column names
    print(f"Column Names: {selected_df.columns.tolist()}\n")
    
    # Print data types of columns
    print(f"Data Types:\n{selected_df.dtypes}\n")
    
    # Print summary statistics
    print(f"Summary Statistics:\n{selected_df.describe()}\n")
    
    # Print missing values count
    print(f"Missing Values Count:\n{selected_df.isnull().sum()}\n")
    
    # Print the DataFrame head
    print(f"DataFrame Head:\n{selected_df.head()}\n")

# Assuming 'df_AR' is the DataFrame you want to inspect
# For example: df_AR = pd.read_csv('AR.csv')

dataframe_info(df_AR)

Shape: (36916, 8)

Column Names: ['WBS1', 'WBS2', 'WBS3', 'Invoice', 'Period', 'InvoiceDate', 'PaidPeriod', 'InvBalanceSourceCurrency']

Data Types:
WBS1                                object
WBS2                                object
WBS3                                object
Invoice                             object
Period                               int64
InvoiceDate                 datetime64[ns]
PaidPeriod                           int64
InvBalanceSourceCurrency           float64
dtype: object

Summary Statistics:
              Period     PaidPeriod  InvBalanceSourceCurrency
count   36916.000000   36916.000000              36916.000000
mean   201192.436938  205090.196446                 58.709153
std      1591.900228   55490.847166                960.737234
min         0.000000  200002.000000              -9500.000000
25%    200710.000000  200801.000000                  0.000000
50%    201206.000000  201208.000000                  0.000000
75%    201703.000000  201705.000000   

In [9]:
# Define the options for the dropdown
options = ['AR', 'CL', 'EMProjectAssoc', 'LD', 'LedgerAR', 'Opportunity',
           'OpportunityCustomTabFields', 'PR', 'Projects_BillingProjections',
           'Projects_ProjectSetu']

# Create a dropdown widget
dropdown = widgets.Dropdown(options=options, description='Select Option:')

# Define the content for each option
content = {
    'AR': '<h2>Accounts Receivable</h2><table><tbody><tr><td valign="TOP" width="20%"><b><font size="2" face="Arial">ColumnName</font></b></td><td valign="TOP" width="20%"><b><font size="2" face="Arial">Domain</font></b></td><td valign="TOP" width="18%"><b><font size="2" face="Arial">Datatype</font></b></td><td valign="TOP" width="7%"><b><font size="2" face="Arial">NULL</font></b></td><td valign="TOP" width="46%"><b><font size="2" face="Arial">Definition</font></b></td></tr><tr><td valign="TOP"><font size="2" face="Arial">WBS1</font></td><td valign="TOP"><font size="2" face="Arial"></font></td><td valign="TOP"><font size="2" face="Arial">varchar(30)</font></td><td valign="TOP"><font size="2" face="Arial">NO</font></td><td valign="TOP"><font size="2" face="Arial"><pre><font size="2" face="Arial">Number of the project associated with the invoice.</font></pre></font></td></tr><tr><td valign="TOP"><font size="2" face="Arial">WBS2</font></td><td valign="TOP"><font size="2" face="Arial"></font></td><td valign="TOP"><font size="2" face="Arial">varchar(7)</font></td><td valign="TOP"><font size="2" face="Arial">NO</font></td><td valign="TOP"><font size="2" face="Arial"><pre><font size="2" face="Arial">The reference field for the project phase. This column is populated by Vision once the WBS1 column is filled in.</font></pre></font></td></tr><tr><td valign="TOP"><font size="2" face="Arial">WBS3</font></td><td valign="TOP"><font size="2" face="Arial"></font></td><td valign="TOP"><font size="2" face="Arial">varchar(7)</font></td><td valign="TOP"><font size="2" face="Arial">NO</font></td><td valign="TOP"><font size="2" face="Arial"><pre><font size="2" face="Arial">The reference field for the project task. This column is populated by Vision once the WBS1 column is filled in.</font></pre></font></td></tr><tr><td valign="TOP"><font size="2" face="Arial">Invoice</font></td><td valign="TOP"><font size="2" face="Arial"></font></td><td valign="TOP"><font size="2" face="Arial">varchar(12)</font></td><td valign="TOP"><font size="2" face="Arial">NO</font></td><td valign="TOP"><font size="2" face="Arial"><pre><font size="2" face="Arial">The invoice number.</font></pre></font></td></tr><tr><td valign="TOP"><font size="2" face="Arial">Period</font></td><td valign="TOP"><font size="2" face="Arial"></font></td><td valign="TOP"><font size="2" face="Arial">int</font></td><td valign="TOP"><font size="2" face="Arial">NO</font></td><td valign="TOP"><font size="2" face="Arial"><pre><font size="2" face="Arial">Accounting period when the invoice was posted.</font></pre></font></td></tr><tr><td valign="TOP"><font size="2" face="Arial">InvoiceDate</font></td><td valign="TOP"><font size="2" face="Arial"></font></td><td valign="TOP"><font size="2" face="Arial">datetime</font></td><td valign="TOP"><font size="2" face="Arial">YES</font></td><td valign="TOP"><font size="2" face="Arial"><pre><font size="2" face="Arial">Date when the invoice was posted.</font></pre></font></td></tr><tr><td valign="TOP"><font size="2" face="Arial">PaidPeriod</font></td><td valign="TOP"><font size="2" face="Arial"></font></td><td valign="TOP"><font size="2" face="Arial">int</font></td><td valign="TOP"><font size="2" face="Arial">NO</font></td><td valign="TOP"><font size="2" face="Arial"><pre><font size="2" face="Arial">Accounting period when the invoice was fully paid.</font></pre></font></td></tr><tr><td valign="TOP"><font size="2" face="Arial">LinkCompany</a></font></td><td valign="TOP"><font size="2" face="Arial"></font></td><td valign="TOP"><font size="2" face="Arial">varchar(14)</font></td><td valign="TOP"><font size="2" face="Arial">YES</font></td><td valign="TOP"><font size="2" face="Arial"><pre><font size="2" face="Arial">For transactions to be posted to intercompany accounts, the Due From company.</font></pre></font></td></tr><tr><td valign="TOP"><font size="2" face="Arial">InvBalanceSourceCurrency</font></td><td valign="TOP"><font size="2" face="Arial"></font></td><td valign="TOP"><font size="2" face="Arial">decimal(19, 4)</font></td><td valign="TOP"><font size="2" face="Arial">NO</font></td><td valign="TOP"><font size="2" face="Arial"><pre><font size="2" face="Arial"></font></pre></font></td></tr><tr><td valign="TOP"><font size="2" face="Arial">DueDate</font></td><td valign="TOP"><font size="2" face="Arial"></font></td><td valign="TOP"><font size="2" face="Arial">datetime</font></td><td valign="TOP"><font size="2" face="Arial">YES</font></td><td valign="TOP"><font size="2" face="Arial"><pre><font size="2" face="Arial">Due Date</font></pre></font></td></tr><tr><td valign="TOP"><font size="2" face="Arial">RetainageDate</font></td><td valign="TOP"><font size="2" face="Arial"></font></td><td valign="TOP"><font size="2" face="Arial">datetime</font></td><td valign="TOP"><font size="2" face="Arial">YES</font></td><td valign="TOP"><font size="2" face="Arial"><pre><font size="2" face="Arial">The date of the most recent retainage billing.</font></pre></font></td></tr></tbody></table>',
    'CL': "<h2>Client List</h2><table><tbody><tr><td>ColumnName</td><td>Domain</td><td>Datatype</td><td>NULL</td><td>Definition</td></tr><tr><td>ClientID</td><td></td><td>varchar(32)</td><td>NO</td><td><pre>Internal identifier for the client.</pre></td></tr><tr><td>Client</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>Client number.</pre></td></tr><tr><td>Name</td><td></td><td>varchar(100)</td><td>YES</td><td><pre>Client name.</pre></td></tr><tr><td>Type</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>The client Type.</pre></td></tr><tr><td>Status</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>The Client's Status.</pre></td></tr><tr><td>ExportInd</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Reserved for future use.</pre></td></tr><tr><td>WebSite</td><td></td><td>varchar(255)</td><td>YES</td><td><pre>The Client's Web site address.</pre></td></tr><tr><td>Memo</td><td></td><td>varchar(4000)</td><td>YES</td><td><pre>Memo field for notes regarding the client.</pre></td></tr><tr><td>CurrentStatus</td><td></td><td>varchar(50)</td><td>YES</td><td><pre>The Client's current relationship to the Organization.</pre></td></tr><tr><td>CustomCurrencyCode</td><td></td><td>varchar(3)</td><td>YES</td><td><pre>The currency code of any custom currency fields in the Client Info Center.</pre></td></tr><tr><td>ClientInd</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Flag indicating whether the Client option is selected on the General tab.</pre></td></tr><tr><td>VendorInd</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Flag indicating whether the Vendor option is selected on the General tab.</pre></td></tr><tr><td>LinkedVendor</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>If VendorInd is Y, the vendor record selected in the Relationship field.</pre></td></tr><tr><td>PriorWork</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>In the LinkedVendor record, flag indicating whether the Prior Work option is selected.</pre></td></tr><tr><td>Recommend</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>In the LinkedVendor record, flag indicating whether the Recommend option is selected.</pre></td></tr><tr><td>DisadvBusiness</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>In the LinkedVendor record, flag indicating whether the Disadvantaged Business option is selected.</pre></td></tr><tr><td>DisabledVetOwnedSmallBusiness</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>In the LinkedVendor record, flag indicating whether the Disabled Vet. Owned Small Business option is selected.</pre></td></tr><tr><td>HBCU</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>In the LinkedVendor record, flag indicating whether the HBCU option is selected.</pre></td></tr><tr><td>MinorityBusiness</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>In the LinkedVendor record, flag indicating whether the Minority Business option is selected.</pre></td></tr><tr><td>SmallBusiness</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>In the LinkedVendor record, flag indicating whether the Small Business option is selected.</pre></td></tr><tr><td>VetOwnedSmallBusiness</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>In the LinkedVendor record, flag indicating whether the Veteran Owner Small Business option is selected.</pre></td></tr><tr><td>WomanOwned</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>In the LinkedVendor record, flag indicating whether the Woman Owned option is selected.</pre></td></tr><tr><td>AlaskaNative</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>In the LinkedVendor record, flag indicating whether the Alaska Native option is selected.</pre></td></tr><tr><td>SpecialtyType</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>In the LinkedVendor record, the value selected in the Specialty field.</pre></td></tr><tr><td>Specialty</td><td></td><td>varchar(4000)</td><td>YES</td><td><pre>In the LinkedVendor record, the text entered in the Specialty Description field.</pre></td></tr><tr><td>ParentID</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>The client selected in the Parent field.</pre></td></tr><tr><td>ParentLevel1</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>The user-entered Description for the second row in the Parent Levels grid of Configuration, General, Client Hierarchy.</pre></td></tr><tr><td>ParentLevel2</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>The user-entered Description for the third row in the Parent Levels grid of Configuration, General, Client Hierarchy.</pre></td></tr><tr><td>ParentLevel3</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>The user-entered Description for the fourth row in the Parent Levels grid of Configuration, General, Client Hierarchy.</pre></td></tr><tr><td>ParentLevel4</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>The user-entered Description for the fifth row in the Parent Levels grid of Configuration, General, Client Hierarchy.</pre></td></tr><tr><td>Employees</td><td></td><td>int</td><td>NO</td><td><pre>This is the number of employees that the client has.</pre></td></tr><tr><td>AnnualRevenue</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>This is the number of employees that the client has.</pre></td></tr><tr><td>GovernmentAgency</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>A checkbox to indicate that this client record is a government agency.</pre></td></tr><tr><td>Competitor</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>A checkbox to indicate that this client record is a competitor to your firm.</pre></td></tr><tr><td>EightA</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>A checkbox to indicate whether this client qualifies for the socioeconomic status of 8(a).</pre></td></tr><tr><td>Hubzone</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>A checkbox to indicate whether this client qualifies for the socioeconomic status of HUBZone (Historically Underutilized Business Zone).</pre></td></tr><tr><td>IQID</td><td></td><td>varchar(50)</td><td>YES</td><td><pre>ID from IQ system for imported Clients</pre></td></tr><tr><td>Incumbent</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Whether or not Client is an Incumbent from IQ system</pre></td></tr><tr><td>AjeraSync</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Ajera Sync</pre></td></tr><tr><td>TLInternalKey</td><td></td><td>varchar(40)</td><td>YES</td><td><pre>The internal ID for TrafficLIVE</pre></td></tr><tr><td>TLSyncModDate</td><td></td><td>datetime</td><td>YES</td><td><pre></pre></td></tr><tr><td>CreateUser</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The username of the user logged in when the record was created.</pre></td></tr><tr><td>CreateDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The date and time when the record was created.</pre></td></tr><tr><td>ModUser</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The username of the user logged in when the record was modified.</pre></td></tr><tr><td>ModDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The date and time when the record was modified.</pre></td></tr></tbody></table>",
    'EMProjectAssoc': "<h2>Employee Project Associations</h2><table><tbody><tr><td>ColumnName</td><td>Domain</td><td>Datatype</td><td>NULL</td><td>Definition</td></tr><tr><td>RecordID</td><td></td><td>varchar(32)</td><td>NO</td><td><pre>Primary key.</pre></td></tr><tr><td>WBS1</td><td></td><td>varchar(30)</td><td>NO</td><td><pre>The associated project's project number.</pre></td></tr><tr><td>WBS2</td><td></td><td>varchar(7)</td><td>NO</td><td><pre>The associated project phase's phase number.</pre></td></tr><tr><td>WBS3</td><td></td><td>varchar(7)</td><td>NO</td><td><pre>The associated project task's task number.</pre></td></tr><tr><td>Employee</td><td></td><td>varchar(20)</td><td>NO</td><td><pre>The employee's employee number.</pre></td></tr><tr><td>Role</td><td></td><td>varchar(10)</td><td>NO</td><td><pre>The employee's project role.</pre></td></tr><tr><td>RoleDescription</td><td></td><td>varchar(4000)</td><td>YES</td><td><pre>The Role's description.</pre></td></tr><tr><td>TeamStatus</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>Variable indicating the status of an employee being added to a project team. Valid options are Active (approved) and Proposed (awaiting approval.)</pre></td></tr><tr><td>StartDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The date when an employee is associated with a project/phase/task. This column defaults to today's date if the association is via the Projects grid on the Projects tab of the Employee Info Center or the Employees grid of the Team tab in the Project Info Center, and can be overridden by a user with update access to these grids.  If the association is the result of a timesheet posting, this column is left blank. The StartDate must be less than or equal to the EndDate.</pre></td></tr><tr><td>EndDate</td><td></td><td>datetime</td><td>YES</td><td><pre>User-entered date; the last day when the employee was designated to work on a project. This is used for management tracking; it does not prevent the employee from posting time to the project for a subsequent date. The EndDate must be greater than or equal to the StartDate.</pre></td></tr><tr><td>CRMHours</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The hours that employees worked on projects for Ajera CRM, Vision CRM, and GovWin Capture Management.</pre></td></tr><tr><td>CreateUser</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The username of the user logged in when the record was created.</pre></td></tr><tr><td>CreateDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The date and time when the record was created.</pre></td></tr><tr><td>ModUser</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The username of the user logged in when the record was modified.</pre></td></tr><tr><td>ModDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The date and time when the record was modified.</pre></td></tr></tbody></table>",
    'LD': "<h2>Labor Detail</h2><table><tbody><tr><td>ColumnName</td><td>Domain</td><td>Datatype</td><td>NULL</td><td>Definition</td></tr><tr><td>Period</td><td></td><td>int</td><td>NO</td><td><pre>Accounting period when the labor transaction was posted.</pre></td></tr><tr><td>PostSeq</td><td></td><td>int</td><td>NO</td><td><pre>Posting sequence number.</pre></td></tr><tr><td>PKey</td><td></td><td>varchar(32)</td><td>NO</td><td><pre>Internal primary key.</pre></td></tr><tr><td>WBS1</td><td></td><td>varchar(255)</td><td>YES</td><td><pre>Project number.</pre></td></tr><tr><td>WBS2</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>The reference field for the project phase. This column is populated by Vision once the WBS1 column in this table is filled in.</pre></td></tr><tr><td>WBS3</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>The reference field for the project task. This column is populated by Vision once the WBS1 column in this table is filled in.</pre></td></tr><tr><td>LaborCode</td><td></td><td>varchar(14)</td><td>YES</td><td><pre>Labor code.</pre></td></tr><tr><td>Employee</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>Employee number.</pre></td></tr><tr><td>TransType</td><td></td><td>varchar(2)</td><td>YES</td><td><pre>Transaction type.</pre></td></tr><tr><td>TransDate</td><td></td><td>datetime</td><td>YES</td><td><pre>Date when the labor transaction was posted.</pre></td></tr><tr><td>Name</td><td></td><td>varchar(100)</td><td>YES</td><td><pre>Employee name.</pre></td></tr><tr><td>RegHrs</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Regular hours posted.</pre></td></tr><tr><td>OvtHrs</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Overtime hours posted.</pre></td></tr><tr><td>RegAmt</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Regular amount posted.</pre></td></tr><tr><td>OvtAmt</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Overtime amount posted.</pre></td></tr><tr><td>BillExt</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Extended billing amount entered.</pre></td></tr><tr><td>Rate</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Employee's rate.</pre></td></tr><tr><td>OvtPct</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Employee's overtime percent.</pre></td></tr><tr><td>OvtRate</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Employee's overtime rate.</pre></td></tr><tr><td>EmType</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>Variable indicating the employee type. Valid options are P-Principal, E-Employee.</pre></td></tr><tr><td>Pool</td><td></td><td>smallint</td><td>NO</td><td><pre>Reserved for future use.</pre></td></tr><tr><td>Category</td><td></td><td>smallint</td><td>NO</td><td><pre>Default billing category for the By Category labor billing method.</pre></td></tr><tr><td>EmOrg</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>Employee's organization.</pre></td></tr><tr><td>PrOrg</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>Project's organization.</pre></td></tr><tr><td>ChargeType</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>Variable indicating the project's charge type. Valid options are R-Regular, H-Overhead, P-Promotional.</pre></td></tr><tr><td>RateType</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>Variable indicating the employee's rate type. Valid options are H-Hourly, S-Salaried.</pre></td></tr><tr><td>DebitLedgerMiscPKey</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>Internal primary key.</pre></td></tr><tr><td>CreditLedgerMiscPKey</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>Internal primary key.</pre></td></tr><tr><td>SuppressBill</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Variable indicating whether to suppress billing for the transaction.</pre></td></tr><tr><td>BillStatus</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>Variable indicating the billing status for the labor transaction. Valid options are B-Billable (released and available for billing to the client); D - Marked for deletion; F - Final billed (already billed to the client); H - Held (to temporarily prevent billing to the client); M - Modified (changed in some way from the original); N - Not billable; O - Deleted; R - Partial hold/released (part of the transaction has already been held or released); T - Transferred (moved from one project, phase, task, or labor code to another); W - To be written off (transaction has been written off, but the invoice has not been posted); X - Written off (transaction has been written off, and the invoice has been posted.)</pre></td></tr><tr><td>Comment</td><td></td><td>varchar(4000)</td><td>YES</td><td><pre>Comment.</pre></td></tr><tr><td>BilledWBS1</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>Project with which the labor detail was final billed.</pre></td></tr><tr><td>BilledWBS2</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>The reference field for the project phase. This column is populated by Vision once the BilledWBS1 column in this table is filled in.</pre></td></tr><tr><td>BilledWBS3</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>The reference field for the project task. This column is populated by Vision once the BilledWBS1 column in this table is filled in.</pre></td></tr><tr><td>BilledInvoice</td><td></td><td>varchar(12)</td><td>YES</td><td><pre>Invoice on which the labor detail was final billed.</pre></td></tr><tr><td>BilledPeriod</td><td></td><td>int</td><td>NO</td><td><pre>Period when the labor detail was final billed.</pre></td></tr><tr><td>XferWBS1</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>Project from which the labor detail was transferred.</pre></td></tr><tr><td>XferWBS2</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>The reference field for the project phase. This column is populated by Vision once the XferWBS1 column in this table is filled in.</pre></td></tr><tr><td>XferWBS3</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>The reference field for the project task. This column is populated by Vision once the XferWBS1 column in this table is filled in.</pre></td></tr><tr><td>XferLaborCode</td><td></td><td>varchar(14)</td><td>YES</td><td><pre>Labor code from which the labor detail was transferred.</pre></td></tr><tr><td>ProjectCost</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Internal flag used for interactive project reporting.</pre></td></tr><tr><td>BillTaxCodeOverride</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>Tax code that will override the standard billing tax code.</pre></td></tr><tr><td>SpecialOvtHrs</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Special overtime hours.</pre></td></tr><tr><td>SpecialOvtAmt</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Special overtime amount.</pre></td></tr><tr><td>SpecialOvtPct</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Special overtime percent.</pre></td></tr><tr><td>SpecialOvtRate</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Special overtime rate.</pre></td></tr><tr><td>SelPeriod</td><td></td><td>int</td><td>NO</td><td><pre>The period for which the regular timesheet hours are included in the payroll run.</pre></td></tr><tr><td>SelPostSeq</td><td></td><td>int</td><td>NO</td><td><pre>The internal posting sequence number for which the regular timesheet hours are included in the payroll run.</pre></td></tr><tr><td>Payrate</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's payroll rate, from either the Employee Info Center or from Accounting, Cost/Pay Rate Tables.</pre></td></tr><tr><td>PayOvtPct</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's payroll overtime percentage, from either the Employee Info Center or from Accounting, Cost/Pay Rate Tables.</pre></td></tr><tr><td>PaySpecialOvtPct</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's special overtime percentage, from either the Employee Info Center or from Accounting, Cost/Pay Rate Tables.</pre></td></tr><tr><td>SelOvtPeriod</td><td></td><td>int</td><td>NO</td><td><pre>The period for which the overtime or special overtime timesheet hours are included in the payroll run.</pre></td></tr><tr><td>SelOvtPostSeq</td><td></td><td>int</td><td>NO</td><td><pre>The internal posting sequence number for which the overtime or special overtime timesheet hours are included in the payroll run.</pre></td></tr><tr><td>WrittenOffPeriod</td><td></td><td>int</td><td>NO</td><td><pre>The fiscal period when a write-off (W) item is changed to final written-off (X) status, and a deleted (D) item is changed to final deleted (O) status.</pre></td></tr><tr><td>Locale</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>The tax locale.</pre></td></tr><tr><td>RegAmtProjectCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's regular pay amount in the functional currency of the company owning the project; phase, if any; and task, if any.</pre></td></tr><tr><td>OvtAmtProjectCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's overtime pay amount in the functional currency of the company owning the project; phase, if any; and task, if any.</pre></td></tr><tr><td>SpecialOvtAmtProjectCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's special overtime pay amount in the functional currency of the company owning the project; phase, if any; and task, if any.</pre></td></tr><tr><td>RateProjectCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's job cost rate in the functional currency of the company owning the project, the project phase, if any, and the project task, if any.</pre></td></tr><tr><td>OvtRateProjectCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's overtime rate in the functional currency of the company owning the project, phase if any, and task if any.</pre></td></tr><tr><td>SpecialOvtRateProjectCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's special overtime rate in the functional currency of the company owning the project, phase if any, and task if any.</pre></td></tr><tr><td>ProjectExchangeInfo</td><td></td><td>varchar(4000)</td><td>YES</td><td><pre>For internal use.</pre></td></tr><tr><td>RegAmtBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's regular pay amount in the billing currency.</pre></td></tr><tr><td>OvtAmtBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's overtime amount in the billing currency, as identified in the project record.  Applies to all but time analysis conversion adjustment transactions.</pre></td></tr><tr><td>SpecialOvtAmtBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's special overtime pay amount in the billing currency.</pre></td></tr><tr><td>RateBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's job cost rate in the billing currency.</pre></td></tr><tr><td>OvtRateBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's overtime rate in the billing currency.</pre></td></tr><tr><td>SpecialOvtRateBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's special overtime rate in the billing currency.</pre></td></tr><tr><td>BillingExchangeInfo</td><td></td><td>varchar(4000)</td><td>YES</td><td><pre>For internal use.</pre></td></tr><tr><td>RegAmtEmployeeCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's regular pay amount in the employee's home currency.</pre></td></tr><tr><td>OvtAmtEmployeeCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's overtime pay amount in the employee's home currency.</pre></td></tr><tr><td>SpecialOvtAmtEmployeeCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's special overtime pay amount in the employee's home currency.</pre></td></tr><tr><td>RateEmployeeCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's job cost rate in the employee's home currency.</pre></td></tr><tr><td>OvtRateEmployeeCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's overtime rate in the employee's home currency.</pre></td></tr><tr><td>SpecialOvtRateEmployeeCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's special overtime rate in the employee's home currency.</pre></td></tr><tr><td>ExchangeInfo</td><td></td><td>varchar(4000)</td><td>YES</td><td><pre>For internal use.</pre></td></tr><tr><td>EMCurrencyCode</td><td></td><td>varchar(3)</td><td>YES</td><td><pre>The associated employee's home currency.</pre></td></tr><tr><td>XferCategory</td><td></td><td>smallint</td><td>NO</td><td><pre>The billing category for billing transferred labor transactions.</pre></td></tr><tr><td>CostRateTableUsed</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Flag indicating whether a cost rate table was used to calculate the transaction's cost rate.</pre></td></tr><tr><td>BillTax2CodeOverride</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>The secondary tax code that will override the standard billing secondary tax code.</pre></td></tr><tr><td>ModUser</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The user who modified this record.</pre></td></tr><tr><td>AuthorizedBy</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The employee who approved this transaction.</pre></td></tr><tr><td>RealizationAmountEmployeeCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Labor realization amount in the employee currency.</pre></td></tr><tr><td>RealizationAmountProjectCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Labor realization amount in the project currency.</pre></td></tr><tr><td>RealizationAmountBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Labor realization amount in the billing currency.</pre></td></tr><tr><td>NonBill</td><td></td><td>varchar(1)</td><td>YES</td><td><pre></pre></td></tr><tr><td>InvoiceStatus</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>Invoice approval status.</pre></td></tr><tr><td>TimekeeperEndDate</td><td></td><td>datetime</td><td>YES</td><td><pre>End date of the linked timesheet entry.</pre></td></tr><tr><td>TransferredPeriod</td><td></td><td>int</td><td>NO</td><td><pre>The first recurrence date for a managed recurring transaction file.</pre></td></tr><tr><td>TransferredBillStatus</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>This field indicates that the transaction file is a managed recurring transaction file.</pre></td></tr><tr><td>TLInternalKey</td><td></td><td>varchar(40)</td><td>YES</td><td><pre>Traffic Live Internal Key</pre></td></tr><tr><td>TLProcessed</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>The field that indicates that the entry has been processed by TrafficLIVE. For internal use.</pre></td></tr></tbody></table>",
    'LedgerAR': "<h2>AR Ledger - Expense Detail</h2><table><tbody><tr><td>ColumnName</td><td>Domain</td><td>Datatype</td><td>NULL</td><td>Definition</td></tr><tr><td>Period</td><td></td><td>int</td><td>NO</td><td><pre>Fiscal year and period for the expense detail.</pre></td></tr><tr><td>PostSeq</td><td></td><td>int</td><td>NO</td><td><pre>Posting sequence number.</pre></td></tr><tr><td>PKey</td><td></td><td>varchar(32)</td><td>NO</td><td><pre>Internal primary key.</pre></td></tr><tr><td>WBS1</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>The project associated with the transaction.</pre></td></tr><tr><td>WBS2</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>The reference field for the project phase. This column is populated by Vision once the WBS1 column in this table is filled in.</pre></td></tr><tr><td>WBS3</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>The reference field for the project task. This column is populated by Vision once the WBS1 column in this table is filled in.</pre></td></tr><tr><td>Account</td><td></td><td>varchar(13)</td><td>YES</td><td><pre>Account number.</pre></td></tr><tr><td>Org</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>The organization associated with the transaction.</pre></td></tr><tr><td>TransType</td><td></td><td>varchar(2)</td><td>YES</td><td><pre>Transaction type. Valid options are CR - cash receipt; IN - invoice.</pre></td></tr><tr><td>SubType</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>Transaction subtype. Valid options are CR: R - retainage, T - tax (VAT) line item, X - adjustment related to Deltek CFMS conversion; IN: I - interest, R - retainage, X - adjustment related to Deltek CFMS conversion.</pre></td></tr><tr><td>RefNo</td><td></td><td>varchar(12)</td><td>YES</td><td><pre>Reference number associated with the transaction during data entry.</pre></td></tr><tr><td>TransDate</td><td></td><td>datetime</td><td>YES</td><td><pre>Date of the transaction.</pre></td></tr><tr><td>Desc1</td><td></td><td>varchar(80)</td><td>YES</td><td><pre>First description for the transaction.</pre></td></tr><tr><td>Desc2</td><td></td><td>varchar(80)</td><td>YES</td><td><pre>Second description for the transaction.</pre></td></tr><tr><td>Amount</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Transaction amount.</pre></td></tr><tr><td>CBAmount</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Transaction amount on a cash basis.</pre></td></tr><tr><td>BillExt</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The billing extension for project reporting at billing rates.</pre></td></tr><tr><td>ProjectCost</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Internal flag for project reporting.</pre></td></tr><tr><td>AutoEntry</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Variable indicating whether the transaction resulted from an implicit posting.</pre></td></tr><tr><td>SuppressBill</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Variable indicating whether you want to suppress billing for the transaction.</pre></td></tr><tr><td>BillStatus</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>Variable indicating the billing status for the expense transaction. Valid options are B-To be billed, W-To be written off, H-Held, F-Final billed, X-Written off, T-Transferred, I-Inserted, M-Modified, D - To be deleted, N - Not billable, O - Deleted, R - Partial hold/Released.</pre></td></tr><tr><td>SkipGL</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Variable indicating whether the transaction should be included for general ledger reporting. History transactions are not included.</pre></td></tr><tr><td>BankCode</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>Bank code assigned to the transaction during data entry.</pre></td></tr><tr><td>Invoice</td><td></td><td>varchar(12)</td><td>YES</td><td><pre>For cash receipt transactions, the number of the invoice being paid by the transaction.</pre></td></tr><tr><td>InvoiceSection</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>The transaction's corresponding invoice section.</pre></td></tr><tr><td>Employee</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The Employee number.</pre></td></tr><tr><td>Vendor</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The vendor number.</pre></td></tr><tr><td>Line</td><td></td><td>smallint</td><td>NO</td><td><pre>The voucher line number.</pre></td></tr><tr><td>PartialPayment</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Partial payment amount in the voucher payment record.</pre></td></tr><tr><td>Discount</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The discount amount in the voucher payment record.</pre></td></tr><tr><td>Voucher</td><td></td><td>varchar(12)</td><td>YES</td><td><pre>The voucher number.</pre></td></tr><tr><td>BilledWBS1</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>The project on whose invoice the record was final billed.</pre></td></tr><tr><td>BilledWBS2</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>The reference field for the project phase. This column is populated by Vision once the BilledWBS1 column in this table is filled in.</pre></td></tr><tr><td>BilledWBS3</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>The reference field for the project task. This column is populated by Vision once the BilledWBS1 column in this table is filled in.</pre></td></tr><tr><td>BilledInvoice</td><td></td><td>varchar(12)</td><td>YES</td><td><pre>Invoice on which the record was final billed.</pre></td></tr><tr><td>BilledPeriod</td><td></td><td>int</td><td>NO</td><td><pre>Period when the record was final billed.</pre></td></tr><tr><td>Unit</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>Unit number.</pre></td></tr><tr><td>UnitTable</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>Unit table.</pre></td></tr><tr><td>UnitQuantity</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Number of units.</pre></td></tr><tr><td>UnitCostRate</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Cost rate per unit.</pre></td></tr><tr><td>UnitBillingRate</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Billing rate per unit.</pre></td></tr><tr><td>UnitBillExt</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Total billing amount, calculated as the product of UnitBillingRate and UnitQuantity.</pre></td></tr><tr><td>XferWBS1</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>Project from which the record was transferred.</pre></td></tr><tr><td>XferWBS2</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>The reference field for the project phase. This column is populated by Vision once the XferWBS1 column in this table is filled in.</pre></td></tr><tr><td>XferWBS3</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>The reference field for the project task. This column is populated by Vision once the XferWBS1 column in this table is filled in.</pre></td></tr><tr><td>XferAccount</td><td></td><td>varchar(13)</td><td>YES</td><td><pre>Accouont from which the record was transferred.</pre></td></tr><tr><td>TaxCode</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>Tax code to associate with the individual line item.</pre></td></tr><tr><td>TaxBasis</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Tax basis, if any, associated with this transaction.</pre></td></tr><tr><td>TaxCBBasis</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Cash basis tax basis, if any, associated with this transaction.</pre></td></tr><tr><td>BillTaxCodeOverride</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>Tax code that overrides the standard billing tax code.</pre></td></tr><tr><td>WrittenOffPeriod</td><td></td><td>int</td><td>NO</td><td><pre>The accounting period that was active when this record was written off (billing status \"X.\")</pre></td></tr><tr><td>TransactionAmount</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The transaction amount.</pre></td></tr><tr><td>TransactionCurrencyCode</td><td></td><td>varchar(3)</td><td>YES</td><td><pre>The transaction's currency code.</pre></td></tr><tr><td>ExchangeInfo</td><td></td><td>varchar(4000)</td><td>YES</td><td><pre>For internal use.</pre></td></tr><tr><td>AmountProjectCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The transaction amount in the project currency.</pre></td></tr><tr><td>ProjectExchangeInfo</td><td></td><td>varchar(4000)</td><td>YES</td><td><pre>For internal use.</pre></td></tr><tr><td>AmountBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The transaction amount in the billing currency.</pre></td></tr><tr><td>BillingExchangeInfo</td><td></td><td>varchar(4000)</td><td>YES</td><td><pre>For internal use.</pre></td></tr><tr><td>AutoEntryAmount</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>For internal use.</pre></td></tr><tr><td>AutoEntryExchangeInfo</td><td></td><td>varchar(4000)</td><td>YES</td><td><pre>For internal use.</pre></td></tr><tr><td>AutoEntryOrg</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>For internal use.</pre></td></tr><tr><td>AutoEntryAccount</td><td></td><td>varchar(13)</td><td>YES</td><td><pre>For internal use.</pre></td></tr><tr><td>AmountSourceCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The transaction amount in the billing currency for invoices and receipts.</pre></td></tr><tr><td>SourceExchangeInfo</td><td></td><td>varchar(4000)</td><td>YES</td><td><pre>For internal use.</pre></td></tr><tr><td>PONumber</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>The purchase order number.</pre></td></tr><tr><td>UnitCostRateBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The UnitCostRate in the billing currency.</pre></td></tr><tr><td>LinkCompany</td><td></td><td>varchar(14)</td><td>YES</td><td><pre>The Due To or Due From company, for transactions posted to intercompany accounts.</pre></td></tr><tr><td>LinkWBS1</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>For transactions posted or to be posted to intercompany accounts, the Due To or Due From project.</pre></td></tr><tr><td>LinkWBS2</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>For transactions posted or to be posted to intercompany accounts, the Due To or Due From phase.</pre></td></tr><tr><td>LinkWBS3</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>For transactions posted or to be posted to intercompany accounts, the Due To or Due From task.</pre></td></tr><tr><td>BillTax2CodeOverride</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>The secondary tax code that will override the standard secondary tax code.</pre></td></tr><tr><td>GainsAndLossesType</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>For internal use.</pre></td></tr><tr><td>AmountTaxCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The Amount expressed in the transaction's tax currency.</pre></td></tr><tr><td>CBAmountTaxCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The CBAmount expressed in the transaction's tax currency.</pre></td></tr><tr><td>TaxBasisTaxCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The TaxBasis amount expressed in the transaction's tax currency.</pre></td></tr><tr><td>TaxCBBasisTaxCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The TaxCBBasis amount expressed in the transaction's tax currency.</pre></td></tr><tr><td>TaxBasisFunctionalCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The TaxBasis amount in the transaction's functional currency.</pre></td></tr><tr><td>TaxCBBasisFunctionalCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The TaxCBBasis amount in the transaction's functional currency.</pre></td></tr><tr><td>DiscountFunctionalCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre></pre></td></tr><tr><td>ModUser</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The user who modified this record.</pre></td></tr><tr><td>AuthorizedBy</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The Employee ID of the employee who approved this transaction.</pre></td></tr><tr><td>RealizationAmountEmployeeCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's realization amount in the employee currency.</pre></td></tr><tr><td>RealizationAmountProjectCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's realization amount in the project currency.</pre></td></tr><tr><td>RealizationAmountBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The employee's realization amount in the billing currency.</pre></td></tr><tr><td>NonBill</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>This is used for the Invoice Transaction Detail Report to indicate non-billable items.</pre></td></tr><tr><td>CreditMemoRefNo</td><td></td><td>varchar(12)</td><td>YES</td><td><pre>The credit memo reference number.</pre></td></tr><tr><td>OriginalAmountSourceCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Original Amount Source Currency</pre></td></tr><tr><td>OriginalPaymentCurrencyCode</td><td></td><td>varchar(3)</td><td>YES</td><td><pre>Original Payment Currency Code</pre></td></tr><tr><td>InProcessAccount</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Indicator of in-process account entry.</pre></td></tr><tr><td>InProcessAccountCleared</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Indicator of in-process account cleared entry.</pre></td></tr><tr><td>EKOriginalLine</td><td></td><td>smallint</td><td>NO</td><td><pre>The original line number of this transaction, used to tie a billing transfer back to the expense \"reason.\"</pre></td></tr><tr><td>InvoiceStatus</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>Invoice approval status.</pre></td></tr><tr><td>OriginatingVendor</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The actual vendor for this line item, if different from the vendor being paid.</pre></td></tr><tr><td>Diary</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>Diary</pre></td></tr><tr><td>DiaryNo</td><td></td><td>int</td><td>NO</td><td><pre>Diary Number</pre></td></tr><tr><td>CreditCardPrimaryCode</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>Primary Credit Card Code.</pre></td></tr><tr><td>CreditCardCode</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>Credit Card Code.</pre></td></tr><tr><td>CreditCardPKey</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>For internal use.</pre></td></tr><tr><td>TransferredPeriod</td><td></td><td>int</td><td>NO</td><td><pre>This field indicates that a reversing transaction file will be automatically scheduled for reversal.</pre></td></tr><tr><td>TransferredBillStatus</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>This field indicates that an alert should be sent when a scheduled reversing file is posted.</pre></td></tr><tr><td>PreInvoice</td><td></td><td>varchar(12)</td><td>YES</td><td><pre>This field indicates that the transaction file is a reversing transaction file.</pre></td></tr><tr><td>EquipmentID</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>The internal identifier for the equipment record.</pre></td></tr><tr><td>BookCode</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>The additional book code, used only for additional book entries.</pre></td></tr><tr><td>EmOrg</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>Employee Organization</pre></td></tr></tbody></table>",
    'Opportunity': "<h2>Opportunities</h2><table><tbody><tr><td>ColumnName</td><td>Domain</td><td>Datatype</td><td>NULL</td><td>Definition</td></tr><tr><td>OpportunityID</td><td></td><td>varchar(32)</td><td>NO</td><td><pre>Internal reference number for the opportunity record.</pre></td></tr><tr><td>Opportunity</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>The opportunity number.</pre></td></tr><tr><td>Name</td><td></td><td>varchar(255)</td><td>YES</td><td><pre>The opportunity name.</pre></td></tr><tr><td>Address1</td><td></td><td>varchar(50)</td><td>YES</td><td><pre>The first line of the opportunity address.</pre></td></tr><tr><td>Address2</td><td></td><td>varchar(50)</td><td>YES</td><td><pre>The second line of the opportunity address.</pre></td></tr><tr><td>Address3</td><td></td><td>varchar(50)</td><td>YES</td><td><pre>The third line of the opportunity address.</pre></td></tr><tr><td>City</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>The opportunity's city.</pre></td></tr><tr><td>State</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>The opportunity's state.</pre></td></tr><tr><td>Zip</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>The opportunity's zip code.</pre></td></tr><tr><td>Country</td><td></td><td>varchar(2)</td><td>YES</td><td><pre>The opportunity's country.</pre></td></tr><tr><td>OpportunityType</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>The opportunity type.</pre></td></tr><tr><td>Stage</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>The opportunity's stage of development.</pre></td></tr><tr><td>Description</td><td></td><td>varchar(4000)</td><td>YES</td><td><pre>The user-entered text describing the opportunity.</pre></td></tr><tr><td>EstStartDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The opportunity's estimated start date.</pre></td></tr><tr><td>EstCompletionDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The opportunity's estimated completion date.</pre></td></tr><tr><td>EstFees</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Reserved for future use.</pre></td></tr><tr><td>EstConstructionCost</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Reserved for future use.</pre></td></tr><tr><td>Revenue</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The revenue which the opportunity will generate.</pre></td></tr><tr><td>Probability</td><td></td><td>smallint</td><td>NO</td><td><pre>The probability that the opportunity will become a project.</pre></td></tr><tr><td>WeightedRevenue</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The product of Revenue and WeightedRevenue.</pre></td></tr><tr><td>CloseDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The date when the opportunity record was closed.</pre></td></tr><tr><td>ClientID</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>The primary client's client number.</pre></td></tr><tr><td>ContactID</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>The primary contact's contact number.</pre></td></tr><tr><td>PRWBS1</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>The project created for the Opportunity.</pre></td></tr><tr><td>OpenDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The date when the opportunity record was opened.</pre></td></tr><tr><td>Source</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>The source of the opportunity.</pre></td></tr><tr><td>Org</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>The Organization associated with the opportunity.</pre></td></tr><tr><td>Status</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>The opportunity's status.</pre></td></tr><tr><td>PRProposalWBS1</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>The project created for the proposal Opportunity.</pre></td></tr><tr><td>ProjMgr</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The employee number of the opportunity's project manager.</pre></td></tr><tr><td>Principal</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The employee number of the opportunity's principal.</pre></td></tr><tr><td>Supervisor</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The employee number of the opportunity's supervisor.</pre></td></tr><tr><td>PublicNoticeDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The user-entered value in block 2 of an SF330 proposal.</pre></td></tr><tr><td>SolicitationNum</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>The user-entered value in block 3 of an SF330 proposal.</pre></td></tr><tr><td>CustomCurrencyCode</td><td></td><td>varchar(3)</td><td>YES</td><td><pre>The currency code of any custom currency fields in the Opportunity Info Center.</pre></td></tr><tr><td>County</td><td></td><td>varchar(50)</td><td>YES</td><td><pre>The opportunity's County of location, as entered on the Location tab.</pre></td></tr><tr><td>PlanID</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>Internal identifier for the plan created from the opportunity.</pre></td></tr><tr><td>LabBillTable</td><td></td><td>int</td><td>NO</td><td><pre>The billing rate table to use for labor.</pre></td></tr><tr><td>LabCostTable</td><td></td><td>int</td><td>NO</td><td><pre>The cost rate table to use for labor.</pre></td></tr><tr><td>OppCurrencyCode</td><td></td><td>varchar(3)</td><td>YES</td><td><pre>The currency code specified on the Service Estimate tab.</pre></td></tr><tr><td>BillingCurrencyCode</td><td></td><td>varchar(3)</td><td>YES</td><td><pre>The billing currency.</pre></td></tr><tr><td>AllocMethod</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>Variable indicating the Allocation Method selected.</pre></td></tr><tr><td>Timescale</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>Variable indicating the Timescale selected.</pre></td></tr><tr><td>KonaSpace</td><td></td><td>int</td><td>NO</td><td><pre>Name of the Kona space that you created for the opportunity.</pre></td></tr><tr><td>ClosedReason</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>Narrative relating to the closing of the opportunity.</pre></td></tr><tr><td>ClosedNotes</td><td></td><td>varchar(4000)</td><td>YES</td><td><pre>Reason why an opportunity record was closed.</pre></td></tr><tr><td>AwardType</td><td></td><td>varchar(15)</td><td>YES</td><td><pre>The source of contract funding for the opportunity.</pre></td></tr><tr><td>Duration</td><td></td><td>varchar(255)</td><td>YES</td><td><pre>The total duration of time that the opportunity is expected to span.</pre></td></tr><tr><td>ContractTypeGovCon</td><td></td><td>varchar(15)</td><td>YES</td><td><pre>The type of opportunity, such as Time and Materials or Task Order.</pre></td></tr><tr><td>CompetitionType</td><td></td><td>varchar(15)</td><td>YES</td><td><pre>The competition type of the opportunity, such as GSA Schedules or Woman Owned Set-Aside.</pre></td></tr><tr><td>MasterContract</td><td></td><td>varchar(15)</td><td>YES</td><td><pre>Master Contract</pre></td></tr><tr><td>Solicitation</td><td></td><td>varchar(255)</td><td>YES</td><td><pre>The RFP number for the opportunity as transferred from GovWin IQ.</pre></td></tr><tr><td>NAICS</td><td></td><td>varchar(15)</td><td>YES</td><td><pre>The North American Industry Classification System (NAICS) code for the Opportunity.</pre></td></tr><tr><td>OurRole</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>Your firm's expected role, such as Prime or Joint Venture, on the contract that is associated with the opportunity.</pre></td></tr><tr><td>IQID</td><td></td><td>varchar(50)</td><td>YES</td><td><pre>ID from IQ system for imported Opportunities</pre></td></tr><tr><td>ServProCode</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>Fee Estimate Service Profile</pre></td></tr><tr><td>FESurchargePct</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Fee Estimate Surcharge Percentage</pre></td></tr><tr><td>FESurcharge</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Fee Estimate Surcharge Amount</pre></td></tr><tr><td>FEAddlExpensesPct</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Fee Estimate Additional Expenses Percentage</pre></td></tr><tr><td>FEAddlExpenses</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Fee Estimate Additional Expenses Amount</pre></td></tr><tr><td>FEOtherPct</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Fee Estimate Other Percentage</pre></td></tr><tr><td>FEOther</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Fee Estimate Other Amount</pre></td></tr><tr><td>IQLastUpdate</td><td></td><td>datetime</td><td>YES</td><td><pre></pre></td></tr><tr><td>MarketingCoordinator</td><td></td><td>varchar(20)</td><td>YES</td><td><pre></pre></td></tr><tr><td>ProposalManager</td><td></td><td>varchar(20)</td><td>YES</td><td><pre></pre></td></tr><tr><td>BusinessDeveloperLead</td><td></td><td>varchar(20)</td><td>YES</td><td><pre></pre></td></tr><tr><td>CreateUser</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The username of the user logged in when the record was created.</pre></td></tr><tr><td>CreateDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The date and time when the record was created.</pre></td></tr><tr><td>ModUser</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The username of the user logged in when the record was modified.</pre></td></tr><tr><td>ModDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The date and time when the record was modified.</pre></td></tr></tbody></table>",
    'OpportunityCustomTabFields': "<h2>Opportunities Custom Fields</h2><table><tbody><tr><td>ColumnName</td><td>Domain</td><td>Datatype</td><td>NULL</td><td>Definition</td></tr><tr><td>OpportunityID</td><td></td><td>varchar(32)</td><td>NO</td><td><pre>Internal reference number for the Opportunity record.</pre></td></tr><tr><td>CreateUser</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The user who created this record.</pre></td></tr><tr><td>CreateDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The date and time when this record was created.</pre></td></tr><tr><td>ModUser</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The user who modified this record.</pre></td></tr><tr><td>ModDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The date and time when this record was modified.</pre></td></tr></tbody></table>",
    'PR': "<h2>Projects</h2><table><tbody><tr><td>ColumnName</td><td>Domain</td><td>Datatype</td><td>NULL</td><td>Definition</td></tr><tr><td>WBS1</td><td></td><td>varchar(30)</td><td>NO</td><td><pre>Project number.</pre></td></tr><tr><td>WBS2</td><td></td><td>varchar(7)</td><td>NO</td><td><pre>The reference field for the project phase. This column is populated by Vision once the WBS1 column in this table is filled in.</pre></td></tr><tr><td>WBS3</td><td></td><td>varchar(7)</td><td>NO</td><td><pre>The reference field for the project task. This column is populated by Vision once the WBS1 column in this table is filled in.</pre></td></tr><tr><td>Name</td><td></td><td>varchar(40)</td><td>YES</td><td><pre>Project name.</pre></td></tr><tr><td>ChargeType</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>Variable indicating the project's charge type. Valid options are R-Regular, H-Overhead, P-Promotional.</pre></td></tr><tr><td>SubLevel</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Flag indicating whether the project has phases and tasks.</pre></td></tr><tr><td>Principal</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>Principal's employee number.</pre></td></tr><tr><td>ProjMgr</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>Project manager's employee number.</pre></td></tr><tr><td>Supervisor</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>Project, phase or task supervisor's employee number.</pre></td></tr><tr><td>ClientID</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>Client number.</pre></td></tr><tr><td>CLAddress</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>Client address code.</pre></td></tr><tr><td>Fee</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Project's total fee.</pre></td></tr><tr><td>ReimbAllow</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Project's reimbursable allowance.</pre></td></tr><tr><td>ConsultFee</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Project's consultant fee.</pre></td></tr><tr><td>BudOHRate</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Project's budgeted overhead rate.</pre></td></tr><tr><td>Status</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>Project's status. Valid options are A-Active, I-Inactive, D-Dormant.</pre></td></tr><tr><td>RevType</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>Project's revenue method.</pre></td></tr><tr><td>MultAmt</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Multiplier or amount to use for the project when running Revenue Generation.</pre></td></tr><tr><td>Org</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>The project's organization.</pre></td></tr><tr><td>UnitTable</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>Default unit table to use for the project.</pre></td></tr><tr><td>StartDate</td><td></td><td>datetime</td><td>YES</td><td><pre>Expected start date for the project.</pre></td></tr><tr><td>EndDate</td><td></td><td>datetime</td><td>YES</td><td><pre>Expected end date for the project.</pre></td></tr><tr><td>PctComp</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Estimated overall percent complete for the project.</pre></td></tr><tr><td>LabPctComp</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Estimated labor percent complete for the project.</pre></td></tr><tr><td>ExpPctComp</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Estimated expense percent complete for the project.</pre></td></tr><tr><td>BillByDefault</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Variable indicating whether certain project expenses are billable to the client. Valid options are Y-Billable, N-Nonbillable, C-Category-specific.</pre></td></tr><tr><td>BillableWarning</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Type of message to appear in Expense when the user charges certain expenses to the project. Valid options are E-Error, W-Warning, N-None.</pre></td></tr><tr><td>Memo</td><td></td><td>varchar(4000)</td><td>YES</td><td><pre>Memo field for comments on the project.</pre></td></tr><tr><td>BudgetedFlag</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Budgeted labor code check for timesheets. Valid options are N-None, W-Warning, E-Error.</pre></td></tr><tr><td>BudgetedLevels</td><td></td><td>varchar(5)</td><td>YES</td><td><pre>The project's budgeted labor code levels; used when budgeted validation is in effect for timesheets.</pre></td></tr><tr><td>BillWBS1</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>Reference project for the Unbilled Detail Report.</pre></td></tr><tr><td>BillWBS2</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>The reference field for the project phase.</pre></td></tr><tr><td>BillWBS3</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>The reference field for the project task.</pre></td></tr><tr><td>XCharge</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Setting to use for cross charges to the project. Valid options are G-Global (the default setting), N-None, P-Project.</pre></td></tr><tr><td>XChargeMethod</td><td></td><td>smallint</td><td>NO</td><td><pre>Cross-charge method for the project.</pre></td></tr><tr><td>XChargeMult</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Cross-charge multiplier for the project.</pre></td></tr><tr><td>Description</td><td></td><td>varchar(255)</td><td>YES</td><td><pre>The project's description.</pre></td></tr><tr><td>Closed</td><td></td><td>int</td><td>NO</td><td><pre>Reserved for future use.</pre></td></tr><tr><td>ReadOnly</td><td></td><td>int</td><td>NO</td><td><pre>Reserved for future use.</pre></td></tr><tr><td>DefaultEffortDriven</td><td></td><td>int</td><td>NO</td><td><pre>Reserved for future use.</pre></td></tr><tr><td>DefaultTaskType</td><td></td><td>int</td><td>NO</td><td><pre>Reserved for future use.</pre></td></tr><tr><td>VersionID</td><td></td><td>int</td><td>NO</td><td><pre>Reserved for future use.</pre></td></tr><tr><td>ContactID</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>The contact record number for the primary contact.</pre></td></tr><tr><td>CLBillingAddr</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The client's billing address.</pre></td></tr><tr><td>LongName</td><td></td><td>varchar(255)</td><td>YES</td><td><pre>The project's long name.</pre></td></tr><tr><td>Address1</td><td></td><td>varchar(50)</td><td>YES</td><td><pre>The first line of the project's address.</pre></td></tr><tr><td>Address2</td><td></td><td>varchar(50)</td><td>YES</td><td><pre>The second line of the project's address.</pre></td></tr><tr><td>Address3</td><td></td><td>varchar(50)</td><td>YES</td><td><pre>The third line of the project's address.</pre></td></tr><tr><td>City</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>The city of the project's address.</pre></td></tr><tr><td>State</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>The state of the project's address.</pre></td></tr><tr><td>Zip</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>The ZIP code of the project's address.</pre></td></tr><tr><td>County</td><td></td><td>varchar(50)</td><td>YES</td><td><pre>The county of the project's address.</pre></td></tr><tr><td>Country</td><td></td><td>varchar(2)</td><td>YES</td><td><pre>The country of the project's address.</pre></td></tr><tr><td>FederalInd</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Variable indicating whether this is a federal project.</pre></td></tr><tr><td>ProjectType</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>The project type.</pre></td></tr><tr><td>Responsibility</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The Organization's project responsibility.</pre></td></tr><tr><td>Referable</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Flag indicating whether the project can be used as a work reference in proposals.</pre></td></tr><tr><td>EstCompletionDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The project's estimated completion date.</pre></td></tr><tr><td>ActCompletionDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The project's actual completion date.</pre></td></tr><tr><td>ContractDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The project contract's award date.</pre></td></tr><tr><td>BidDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The date when the Organization submitted its bid on the project.</pre></td></tr><tr><td>ComplDateComment</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>User-entered comment regarding the completion date.</pre></td></tr><tr><td>FirmCost</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The Organization's total cost for completing the project.</pre></td></tr><tr><td>FirmCostComment</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>User-entered comment regarding the Firm Cost.</pre></td></tr><tr><td>TotalProjectCost</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The total project cost.</pre></td></tr><tr><td>TotalCostComment</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>User-entered comment regarding the Project Cost.</pre></td></tr><tr><td>OpportunityID</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>The project's Opportunity record number.</pre></td></tr><tr><td>ClientConfidential</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>Flag indicating whether the client's name will be used in proposals, or replaced with an alias.</pre></td></tr><tr><td>ClientAlias</td><td></td><td>varchar(100)</td><td>YES</td><td><pre>The client alias to use in proposals if the client is confidential.</pre></td></tr><tr><td>AvailableForCRM</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Variable indicating whether the record is available to CRM users. The default value is Yes.</pre></td></tr><tr><td>ReadyForApproval</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Variable indicating whether the record is available to Accounting users.</pre></td></tr><tr><td>ReadyForProcessing</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Flag indicating whether the record is approved for use in transaction processing.</pre></td></tr><tr><td>BillingClientID</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>The client record number for the billing client.</pre></td></tr><tr><td>BillingContactID</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>The contact record number for the billing contact.</pre></td></tr><tr><td>Phone</td><td></td><td>varchar(24)</td><td>YES</td><td><pre>The project location telephone number.</pre></td></tr><tr><td>Fax</td><td></td><td>varchar(24)</td><td>YES</td><td><pre>The project location facsimile transmission telephone number.</pre></td></tr><tr><td>EMail</td><td></td><td>varchar(50)</td><td>YES</td><td><pre>The project location e-mail address.</pre></td></tr><tr><td>ProposalWBS1</td><td></td><td>varchar(30)</td><td>YES</td><td><pre>The promotional project linked to the record.</pre></td></tr><tr><td>CostRateMeth</td><td></td><td>smallint</td><td>NO</td><td><pre>The project's cost rate method. Valid options are 0 - None; 1 - from Labor Rate Table; 2 - from Category Rate Table; 3 - from Labor Code Rate Table; 4 - from Employee Cost Rate.</pre></td></tr><tr><td>CostRateTableNo</td><td></td><td>int</td><td>NO</td><td><pre>The cost rate table number.</pre></td></tr><tr><td>PayRateMeth</td><td></td><td>smallint</td><td>NO</td><td><pre>The project's pay rate method. Valid options are 0 - None; 1 - from Labor Rate Table; 2 - from Category Rate Table; 3 - from Labor Code Rate Table; 4 - from Employee Pay Rate.</pre></td></tr><tr><td>PayRateTableNo</td><td></td><td>int</td><td>NO</td><td><pre>The pay rate table number.</pre></td></tr><tr><td>Locale</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>The project's payroll tax locale.</pre></td></tr><tr><td>LineItemApproval</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Variable indicating the project's timesheet line item approval setting. Valid options are Yes, No, and System (the default setting.)</pre></td></tr><tr><td>LineItemApprovalEK</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Variable indicating the project's expense report line item approval setting. Valid options are Yes, No, and System (the default setting.)</pre></td></tr><tr><td>BudgetSource</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>This field is active only when BudgetedFlag is set to Error or Warning. The budget to use for budget validation of timesheets. Valid options are Project Planning or Budget Worksheet.</pre></td></tr><tr><td>BudgetLevel</td><td></td><td>varchar(1)</td><td>YES</td><td><pre>This field is active only when the BudgetSource is Project Planning. Valid options are Employee Only, Labor Code Only, or Both.</pre></td></tr><tr><td>ProfServicesComplDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The user-entered value in Prof. Services Compl. Dt. on the project's Background tab; used to auto-populate Part I, block 19 (2) Year Completed - Professional Services of an SF330 proposal.</pre></td></tr><tr><td>ConstComplDate</td><td></td><td>datetime</td><td>YES</td><td><pre>The user-entered value in Cons. Compl. Dt. on the project's Background tab; used to auto-populate Part I, block 19 (2) Year Completed - Construction of an SF330 proposal.</pre></td></tr><tr><td>ProjectCurrencyCode</td><td></td><td>varchar(3)</td><td>YES</td><td><pre>The project's functional currency.</pre></td></tr><tr><td>ProjectExchangeRate</td><td></td><td>decimal(19, 10)</td><td>NO</td><td><pre>An optimal fixed exchange rate to be used for any currency exchanges from the functional currency of the project/phase/task to the project currency.</pre></td></tr><tr><td>BillingCurrencyCode</td><td></td><td>varchar(3)</td><td>YES</td><td><pre>The project's billing currency.</pre></td></tr><tr><td>BillingExchangeRate</td><td></td><td>decimal(19, 10)</td><td>NO</td><td><pre>An optimal fixed exchange rate to be used for any currency exchanges from the functional currency of the project/phase/task to the billing currency.</pre></td></tr><tr><td>RestrictChargeCompanies</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Flag indicating whether charges to the project are restricted to named companies.</pre></td></tr><tr><td>FeeBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The project's fee in the project's billing currency.</pre></td></tr><tr><td>ReimbAllowBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The project's reimbursable allowance in the project's billing currency.</pre></td></tr><tr><td>ConsultFeeBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The project's consultant fee in the project's billing currency.</pre></td></tr><tr><td>RevUpsetLimits</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Flag indicating whether revenue upset limits are enabled for the project.</pre></td></tr><tr><td>RevUpsetWBS2</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>Variable indicating whether revenue upset limits are enabled at the phase level.</pre></td></tr><tr><td>RevUpsetWBS3</td><td></td><td>varchar(7)</td><td>YES</td><td><pre>Variable indicating whether revenue upset limits are enabled at the task level.</pre></td></tr><tr><td>RevUpsetIncludeComp</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Flag indicating whether to include compensation in the project's revenue upset limit.</pre></td></tr><tr><td>RevUpsetIncludeCons</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Flag indicating whether to include consultant expense in the project's revenue upset limit.</pre></td></tr><tr><td>RevUpsetIncludeReimb</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Flag indicating whether to include reimbursable expense in the project's revenue upset limit.</pre></td></tr><tr><td>PORMBRate</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The reimbursable expense markup for the project's committed expenses from purchase orders.</pre></td></tr><tr><td>POCNSRate</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The consultant expense markup for the project's committed expenses from purchase orders.</pre></td></tr><tr><td>PlanID</td><td></td><td>varchar(32)</td><td>YES</td><td><pre>Internal identifier for the Plan record.</pre></td></tr><tr><td>TKCheckRPDate</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Flag indicating whether Vision Time should check task start and end dates in Resource Planning.</pre></td></tr><tr><td>ICBillingLab</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>The Intercompany Labor flag on the project's Intercompany Billing tab. The flag indicates whether to apply the rate method at the Global or Project level.</pre></td></tr><tr><td>ICBillingLabMethod</td><td></td><td>smallint</td><td>NO</td><td><pre>PRTemplate</pre></td></tr><tr><td>ICBillingLabMult</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The multiplier for Intercompany Labor, on the project's Intercompany Billing tab. This multiplier is used when the ICBillingLabMethod is Rate Method - Multiplier.</pre></td></tr><tr><td>ICBillingExp</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>The Intercompany Expense flag, on the project's Intercompany Billing tab. The flag indicates whether to apply the rate method at the Global or Project level.</pre></td></tr><tr><td>ICBillingExpMethod</td><td></td><td>smallint</td><td>NO</td><td><pre>The rate method for Intercompany Expense, on the project's Intercompany Billing tab. The method is used when the Rate Method is applied at the project level. Valid options are Multiplier and Billing Terms.</pre></td></tr><tr><td>ICBillingExpMult</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The multiplier for Intercompany Expense, on the project's Intercompany Billing tab. The multiplier is used when the ICBillingExpMethod is Multiplier.</pre></td></tr><tr><td>RequireComments</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Flag indicating whether the Require Comments for Entered Hours option is selected on the project record's Time &amp; Expense tab.</pre></td></tr><tr><td>TKCheckRPPlannedHrs</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Flag indicating whether the Check Project Planning for Planned Hours option is selected on the project record's Time &amp; Expense tab.</pre></td></tr><tr><td>BillByDefaultConsultants</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Variable indicating the selection in the Bill by Default/Consultants field. Valid options are Y - Yes, N - No, and E - Expense Code.</pre></td></tr><tr><td>BillByDefaultOtherExp</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>Variable indicating the selection in the Bill by Default/Other Expenses field. Valid options are Y - Yes, N - No, and E - Expense Code.</pre></td></tr><tr><td>BillByDefaultORTable</td><td></td><td>int</td><td>NO</td><td><pre>The override table number selected in the Bill by Default/Override Table field.</pre></td></tr><tr><td>PhoneFormat</td><td></td><td>varchar(24)</td><td>YES</td><td><pre>The number format to use for the Phone value.</pre></td></tr><tr><td>FaxFormat</td><td></td><td>varchar(24)</td><td>YES</td><td><pre>The number format to use for the Fax value.</pre></td></tr><tr><td>RevType2</td><td></td><td>varchar(10)</td><td>NO</td><td><pre>The revenue method to use for the second uninvoiced acccount when the revenue by category feature is enabled.</pre></td></tr><tr><td>RevType3</td><td></td><td>varchar(10)</td><td>NO</td><td><pre>The revenue method to use for the third uninvoiced acccount when the revenue by category feature is enabled.</pre></td></tr><tr><td>RevType4</td><td></td><td>varchar(10)</td><td>NO</td><td><pre>The revenue method to use for the fourth uninvoiced acccount when the revenue by category feature is enabled.</pre></td></tr><tr><td>RevType5</td><td></td><td>varchar(10)</td><td>NO</td><td><pre>The revenue method to use for the fifth uninvoiced acccount when the revenue by category feature is enabled.</pre></td></tr><tr><td>RevUpsetCategoryToAdjust</td><td></td><td>smallint</td><td>NO</td><td><pre>The unbillled revenue category in which to place any adjustment when the revenue upset limit is exceeded.</pre></td></tr><tr><td>FeeFunctionalCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The total compensation in the project's functional currency.</pre></td></tr><tr><td>ReimbAllowFunctionalCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The reimbursable allowance in the project's functional currency.</pre></td></tr><tr><td>ConsultFeeFunctionalCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>The consultant fee in the project's functional currency.</pre></td></tr><tr><td>RevenueMethod</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>The project's revenue method.</pre></td></tr><tr><td>ICBillingLabTableNo</td><td></td><td>int</td><td>NO</td><td><pre>The intercompany labor rate table ID for the rate table entered in Project Info Center » Intercompany Billing tab » Intercompany Labor section.</pre></td></tr><tr><td>ICBillingExpTableNo</td><td></td><td>int</td><td>NO</td><td><pre>The intercompany expense rate table ID for the rate table entered in Project Info Center » Intercompany Billing tab » Intercompany Expense section.</pre></td></tr><tr><td>Biller</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>Internal person who is responsible for sending out client invoices.</pre></td></tr><tr><td>FeeDirLab</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre></pre></td></tr><tr><td>FeeDirExp</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre></pre></td></tr><tr><td>ReimbAllowExp</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre></pre></td></tr><tr><td>ReimbAllowCons</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre></pre></td></tr><tr><td>FeeDirLabBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre></pre></td></tr><tr><td>FeeDirExpBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre></pre></td></tr><tr><td>ReimbAllowExpBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre></pre></td></tr><tr><td>ReimbAllowConsBillingCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre></pre></td></tr><tr><td>FeeDirLabFunctionalCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre></pre></td></tr><tr><td>FeeDirExpFunctionalCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre></pre></td></tr><tr><td>ReimbAllowExpFunctionalCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre></pre></td></tr><tr><td>ReimbAllowConsFunctionalCurrency</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre></pre></td></tr><tr><td>RevUpsetIncludeCompDirExp</td><td></td><td>varchar(1)</td><td>NO</td><td><pre></pre></td></tr><tr><td>RevUpsetIncludeReimbCons</td><td></td><td>varchar(1)</td><td>NO</td><td><pre></pre></td></tr><tr><td>AwardType</td><td></td><td>varchar(15)</td><td>YES</td><td><pre>Award Type</pre></td></tr><tr><td>Duration</td><td></td><td>varchar(255)</td><td>YES</td><td><pre>Duration</pre></td></tr><tr><td>ContractTypeGovCon</td><td></td><td>varchar(15)</td><td>YES</td><td><pre>Contract Type GovCon</pre></td></tr><tr><td>CompetitionType</td><td></td><td>varchar(15)</td><td>YES</td><td><pre>Competition Type</pre></td></tr><tr><td>MasterContract</td><td></td><td>varchar(15)</td><td>YES</td><td><pre>Master Contract</pre></td></tr><tr><td>Solicitation</td><td></td><td>varchar(255)</td><td>YES</td><td><pre>Solicitation</pre></td></tr><tr><td>NAICS</td><td></td><td>varchar(15)</td><td>YES</td><td><pre>NAICS</pre></td></tr><tr><td>OurRole</td><td></td><td>varchar(20)</td><td>YES</td><td><pre></pre></td></tr><tr><td>AjeraSync</td><td></td><td>varchar(1)</td><td>NO</td><td><pre>The pre-invoice number.</pre></td></tr><tr><td>ServProCode</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>Fee Estimate Service Profile</pre></td></tr><tr><td>FESurchargePct</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Fee Estimate Surcharge Percentage</pre></td></tr><tr><td>FESurcharge</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Fee Estimate Surcharge Amount</pre></td></tr><tr><td>FEAddlExpensesPct</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Fee Estimate Additional Expenses Percentage</pre></td></tr><tr><td>FEAddlExpenses</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Fee Estimate Additional Expenses Amount</pre></td></tr><tr><td>FEOtherPct</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Fee Estimate Other Percentage</pre></td></tr><tr><td>FEOther</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Fee Estimate Other Amount</pre></td></tr><tr><td>ProjectTemplate</td><td></td><td>varchar(10)</td><td>YES</td><td><pre>The project template code as defined in the Ajera Project Template code table in  Configuration.</pre></td></tr><tr><td>AjeraSpentLabor</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Ajera Spent Labor</pre></td></tr><tr><td>AjeraSpentReimbursable</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Ajera Spent Reimbursable</pre></td></tr><tr><td>AjeraSpentConsultant</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Ajera Spent Consultant</pre></td></tr><tr><td>AjeraCostLabor</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Ajera Cost Labor</pre></td></tr><tr><td>AjeraCostReimbursable</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Ajera Cost Reimbursable</pre></td></tr><tr><td>AjeraCostConsultant</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Ajera Cost Consultant</pre></td></tr><tr><td>AjeraWIPLabor</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Ajera WIP Labor</pre></td></tr><tr><td>AjeraWIPReimbursable</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Ajera WIP Reimbursable</pre></td></tr><tr><td>AjeraWIPConsultant</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Ajera WIP Consultant</pre></td></tr><tr><td>AjeraBilledLabor</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Ajera Billed Labor</pre></td></tr><tr><td>AjeraBilledReimbursable</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Ajera Billed Reimbursable</pre></td></tr><tr><td>AjeraBilledConsultant</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Ajera Billed Consultant</pre></td></tr><tr><td>AjeraReceivedLabor</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Ajera Received Labor</pre></td></tr><tr><td>AjeraReceivedReimbursable</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Ajera Received Reimbursable</pre></td></tr><tr><td>AjeraReceivedConsultant</td><td></td><td>decimal(19, 4)</td><td>NO</td><td><pre>Ajera Received Consultant</pre></td></tr><tr><td>TLInternalKey</td><td></td><td>varchar(40)</td><td>YES</td><td><pre>The internal ID for TrafficLIVE.</pre></td></tr><tr><td>TLProjectID</td><td></td><td>varchar(40)</td><td>YES</td><td><pre>Traffic Live Project ID</pre></td></tr><tr><td>TLProjectName</td><td></td><td>varchar(255)</td><td>YES</td><td><pre>Traffic Live Project Name</pre></td></tr><tr><td>TLChargeBandInternalKey</td><td></td><td>varchar(40)</td><td>YES</td><td><pre>The Chargeband internal id for TrafficLIVE.</pre></td></tr><tr><td>TLChargeBandExternalCode</td><td></td><td>varchar(14)</td><td>YES</td><td><pre>The Chargeband (mapped to Level 1 Labor Code) for TrafficLIVE.</pre></td></tr><tr><td>TLSyncModDate</td><td></td><td>datetime</td><td>YES</td><td><pre></pre></td></tr><tr><td>CreateUser</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>Username of the user logged in when this record was created.</pre></td></tr><tr><td>CreateDate</td><td></td><td>datetime</td><td>YES</td><td><pre>Date and time when this record was created.</pre></td></tr><tr><td>ModUser</td><td></td><td>varchar(20)</td><td>YES</td><td><pre>The username of the user logged in when the record was modified.</pre></td></tr><tr><td>ModDate</td><td></td><td>datetime</td><td>YES</td><td><pre>Date and time when this record was modified.</pre></td></tr></tbody></table>",
    'Projects_BillingProjections': "",
    'Projects_ProjectSetu': ""
}

# Define a function to update the displayed content
def update_content(option):
    selected_content = content.get(option, 'Invalid Option')
    display(HTML(selected_content))  # Display HTML content

# Use an observer to update the content when the dropdown selection changes
widgets.interactive(update_content, option=dropdown)

interactive(children=(Dropdown(description='Select Option:', options=('AR', 'CL', 'EMProjectAssoc', 'LD', 'Led…

In [10]:
def process_dataframes(df_names):
    results = {}

    for df_name in df_names:
        # Get the DataFrame
        df = globals()[df_name]

        # Get the total number of rows
        num_rows = len(df)

        # Get a list of columns in the DataFrame
        columns = df.columns

        # Create an empty list to store column names and their respective counts of null values
        null_counts = []

        # Create an empty list to store the names of dropped columns
        dropped_columns = []

        # Iterate over the columns and count null values
        for column in columns:
            null_count = df[column].isna().sum()
            percent_null = (null_count / num_rows) * 100
            null_counts.append((column, null_count, percent_null))

            if percent_null == 100:
                df.drop(column, axis=1, inplace=True)
                dropped_columns.append(column)

        # Store the results for this DataFrame
        df_results = {
            "Initial Columns": len(columns),
            "Final Columns": len(df.columns),
            "Dropped Columns": dropped_columns
        }

        results[df_name] = df_results

    return results

In [11]:
df_names = []
for i in range(0, len(dflist)):
    df_names.append("df_"+dflist[i][0])

In [12]:
process_dataframes(df_names)

{'df_AR': {'Initial Columns': 8, 'Final Columns': 8, 'Dropped Columns': []},
 'df_CL': {'Initial Columns': 45,
  'Final Columns': 41,
  'Dropped Columns': ['ParentLevel4',
   'IQID',
   'TLInternalKey',
   'TLSyncModDate']},
 'df_EMProjectAssoc': {'Initial Columns': 15,
  'Final Columns': 15,
  'Dropped Columns': []},
 'df_LD': {'Initial Columns': 92,
  'Final Columns': 84,
  'Dropped Columns': ['BillTaxCodeOverride',
   'Locale',
   'EMCurrencyCode',
   'BillTax2CodeOverride',
   'ModUser',
   'AuthorizedBy',
   'InvoiceStatus',
   'TLInternalKey']},
 'df_LedgerAR': {'Initial Columns': 104,
  'Final Columns': 72,
  'Dropped Columns': ['Employee',
   'Vendor',
   'Voucher',
   'Unit',
   'TransactionCurrencyCode',
   'AutoEntryExchangeInfo',
   'AutoEntryOrg',
   'AutoEntryAccount',
   'SourceExchangeInfo',
   'PONumber',
   'LinkCompany',
   'LinkWBS1',
   'LinkWBS2',
   'LinkWBS3',
   'BillTax2CodeOverride',
   'GainsAndLossesType',
   'ModUser',
   'AuthorizedBy',
   'NonBill',
   '

In [13]:
# R01.2004.003904
# R01.2004.004101
# R01.2004.004138
# R01.2004.004167
# R01.2005.004307
# R05.2015.986001    5934
# R05.2015.000986    4270
# R01.2009.006205    2939
# R01.2003.003395    2640
# R01.2001.002776    2549
# df_LedgerAR.loc[df_LedgerAR['WBS1'] == 'R05.2015.986001']
df_AR.loc[df_AR['WBS1'] == 'R05.2015.986001']

,WBS1,WBS2,WBS3,Invoice,Period,InvoiceDate,PaidPeriod,InvBalanceSourceCurrency
6141,R05.2015.986001,2,1,25174,202004,2020-04-30,202005,0.0
7495,R05.2015.986001,3,1,25475,202011,2020-11-30,202101,0.0
7496,R05.2015.986001,3,1,25598,202102,2021-02-28,202103,0.0
8520,R05.2015.986001,4,1,25714,202104,2021-04-30,202106,0.0
8521,R05.2015.986001,4,1,25763,202105,2021-05-31,202106,0.0
...,...,...,...,...,...,...,...,...
33738,R05.2015.986001,1,2,25174,202004,2020-04-30,202005,0.0
33739,R05.2015.986001,1,2,25219,202005,2020-05-31,202006,0.0
33740,R05.2015.986001,1,2,25295,202007,2020-07-31,202008,0.0
33741,R05.2015.986001,1,2,25383,202009,2020-09-30,202010,0.0


In [14]:
df_EMProjectAssoc.loc[df_EMProjectAssoc['WBS1'] == 'R05.2015.986001']

,RecordID,WBS1,WBS2,WBS3,Employee,Role,RoleDescription,TeamStatus,StartDate,EndDate,CreateUser,CreateDate,ModUser,ModDate,CRMHours
3022,F97E955C4169472FB813AF380D9F6DB1,R05.2015.986001,NaN,NaN,88,NaN,NaN,Active,NaT,NaT,KOBRIEN,2019-10-01 00:50:16,KOBRIEN,2019-12-06 08:51:04,0
4366,4EE3F8A381B145E48CD5CBC22DC88329,R05.2015.986001,NaN,NaN,114,NaN,NaN,Active,NaT,NaT,KOBRIEN,2019-10-01 00:50:16,KOBRIEN,2020-01-28 10:08:29,0
5090,1A3BBC6BDAF74B1496049DBAAFCA2C15,R05.2015.986001,NaN,NaN,60,NaN,NaN,Active,NaT,NaT,KOBRIEN,2019-10-01 00:50:16,KOBRIEN,2020-04-15 10:56:00,0
6237,0C0AF8AB04114E20A593D8BD2822EDFC,R05.2015.986001,NaN,NaN,194,NaN,NaN,Active,NaT,NaT,KOBRIEN,2019-10-01 00:50:16,KOBRIEN,2020-02-12 16:08:12,0
6930,DB49FBC5C0B84AB1BA1D549F98A66753,R05.2015.986001,NaN,NaN,160,NaN,NaN,Active,NaT,NaT,KOBRIEN,2019-10-01 00:50:16,KOBRIEN,2020-04-15 10:56:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465875,DD81217EE9D24D5AB2E2F8853E55DA23,R05.2015.986001,7,NaN,312,sysST,"<P class=MsoNormal style=""MARGIN: 0in 0in 8pt""...",Proposed,NaT,NaT,KOBRIEN,2019-10-01 01:48:55,KOBRIEN,2019-10-01 01:48:55,0
465893,F6149C11AD00468DA08D5528787F9AA7,R05.2015.986001,8,NaN,304,sysST,"<P class=MsoNormal style=""MARGIN: 0in 0in 8pt""...",Proposed,NaT,NaT,KOBRIEN,2019-10-01 01:49:48,KOBRIEN,2019-10-01 01:49:48,0
465898,FA8B644BBA3249369262EED8FC6AF1E7,R05.2015.986001,1,NaN,304,sysST,"<P class=MsoNormal style=""MARGIN: 0in 0in 8pt""...",Active,NaT,NaT,KOBRIEN,2019-10-01 00:57:57,KOBRIEN,2020-04-29 11:49:45,0
469156,F9CCB630B3D244B5A0A623B7297875E2,R05.2015.986001,NaN,NaN,312,sysST,"<P class=MsoNormal style=""MARGIN: 0in 0in 8pt""...",Proposed,NaT,NaT,KOBRIEN,2019-10-01 00:50:16,KOBRIEN,2019-10-01 00:50:16,0


In [15]:
df_LD.loc[df_LD['WBS1'] == 'R05.2015.986001']

,Period,PostSeq,PKey,WBS1,WBS2,WBS3,LaborCode,Employee,TransType,TransDate,...,XferCategory,CostRateTableUsed,RealizationAmountEmployeeCurrency,RealizationAmountProjectCurrency,RealizationAmountBillingCurrency,NonBill,TimekeeperEndDate,TransferredPeriod,TransferredBillStatus,TLProcessed
265968,202003,86,638985875417,R05.2015.986001,13,5,1000,88,TS,2020-03-09,...,0,N,0.0,0.0,0.0,NaN,NaT,0,NaN,N
265969,202003,86,638985885445,R05.2015.986001,13,5,1000,88,TS,2020-03-10,...,0,N,0.0,0.0,0.0,NaN,NaT,0,NaN,N
265970,202003,86,638985899397,R05.2015.986001,13,5,1000,88,TS,2020-03-11,...,0,N,0.0,0.0,0.0,NaN,NaT,0,NaN,N
265971,202003,86,638985909418,R05.2015.986001,13,5,1000,88,TS,2020-03-12,...,0,N,0.0,0.0,0.0,NaN,NaT,0,NaN,N
265972,202003,86,638985922832,R05.2015.986001,13,5,1000,88,TS,2020-03-13,...,0,N,0.0,0.0,0.0,NaN,NaT,0,NaN,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945859,202107,73,681220980468,R05.2015.986001,1,1,OFFIC,329,TS,2021-07-12,...,0,N,0.0,0.0,0.0,NaN,NaT,0,NaN,N
945860,202107,73,681221199887,R05.2015.986001,1,1,OFFIC,329,TS,2021-07-14,...,0,N,0.0,0.0,0.0,NaN,NaT,0,NaN,N
945861,202107,73,681221233391,R05.2015.986001,1,1,OFFIC,329,TS,2021-07-09,...,0,N,0.0,0.0,0.0,NaN,NaT,0,NaN,N
945862,202107,73,681221233394,R05.2015.986001,1,1,OFFIC,329,TS,2021-07-21,...,0,N,0.0,0.0,0.0,NaN,NaT,0,NaN,N


In [50]:
type(df_LD['RegHrs'][811564]) #int 811555 float - 811564

float

In [68]:
df_LD['RegHrs'][0:5] #[990000:990050]

0    1900-01-09 00:00:00
1    1900-01-10 00:00:00
2    1900-01-08 12:00:00
3    1900-01-08 12:00:00
4    1900-01-07 00:00:00
Name: RegHrs, dtype: object

In [24]:
df_LD["OvtHrs"].value_counts().sum()

1024657

In [16]:
total_hours = len(df_LD)
nonbillable = (df_LD['NonBill'] == 'N').sum() # 19582
meetings = (df_LD['LaborCode'] == 'MEETG').sum() # 87333

In [17]:
meetings

87333

In [18]:
print(nonbillable / total)

0.019110785365249055


In [19]:
print(meetings / total)

0.08523144818217218


In [49]:
df_LedgerAR.loc[df_LedgerAR['WBS1'] == 'R05.2015.986001']

,Period,PostSeq,PKey,WBS1,WBS2,WBS3,Account,Org,TransType,SubType,...,DiaryNo,TransferredPeriod,TransferredBillStatus,BilledInvoice,XferWBS1,XferWBS2,XferWBS3,XferAccount,TaxCode,ExchangeInfo
16079,202001,94,633803379717,R05.2015.986001,1,2,45000,05-FES,IN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16083,202002,99,637067334989,R05.2015.986001,1,2,45000,05-FES,IN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16089,202003,105,639569756811,R05.2015.986001,1,2,45000,05-FES,IN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16097,202004,100,642090122216,R05.2015.986001,1,2,45000,05-FES,IN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16101,202005,97,644602667469,R05.2015.986001,1,2,45000,05-FES,IN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219136,202110,44,688230315037,R05.2015.986001,1,54,40000,05-FES,CR,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219137,202110,44,688230315045,R05.2015.986001,1,32,40000,05-FES,CR,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219138,202110,44,688230315057,R05.2015.986001,1,58,40000,05-FES,CR,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219139,202110,44,688230315067,R05.2015.986001,1,50,40000,05-FES,CR,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df_LedgerAR.loc[df_LedgerAR['WBS1'] == 'R05.2015.986001']['Account'].value_counts()

40000    1942
42000    1676
12000     852
12010     734
45000     367
43000     246
12020     117
Name: Account, dtype: int64

In [53]:
df_PR.loc[df_PR['WBS1'] == 'R05.2015.986001']

,WBS1,WBS2,WBS3,Name,ChargeType,SubLevel,Principal,ProjMgr,Supervisor,ClientID,...,AjeraWIPLabor,AjeraWIPReimbursable,AjeraWIPConsultant,AjeraBilledLabor,AjeraBilledReimbursable,AjeraBilledConsultant,AjeraReceivedLabor,AjeraReceivedReimbursable,AjeraReceivedConsultant,PIMID
16644,R05.2015.986001,6,NaN,"PSS - 5100- Water Treatment System (<20,",R,Y,61,53.0,83.0,NPS,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,R05.2015.986001
16645,R05.2015.986001,7,NaN,"PSS - 5100- Water Treatment System (20,0",R,Y,61,53.0,83.0,NPS,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,R05.2015.986001
16646,R05.2015.986001,8,NaN,PSS - 5100- Water Treatment System (>100,R,Y,61,53.0,83.0,NPS,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,R05.2015.986001
16647,R05.2015.986001,10,NaN,PSS - 5200 - Wastewater Treatment System,R,Y,61,53.0,83.0,NPS,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,R05.2015.986001
16648,R05.2015.986001,11,NaN,PSS - 5200 - Wastewater Treatment System,R,Y,61,53.0,83.0,NPS,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,R05.2015.986001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32552,R05.2015.986001,13,61,FOPU Fort Pulaski NM,R,N,61,53.0,83.0,NPS,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,R05.2015.986001
32553,R05.2015.986001,14,1,KALA Kalaupapa NHP,R,N,61,192.0,83.0,NPS,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,R05.2015.986001
32554,R05.2015.986001,14,2,PAIS Padre Island NS,R,N,61,192.0,83.0,NPS,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,R05.2015.986001
32555,R05.2015.986001,14,3,CUIS/BLRI Work Orders,R,N,61,192.0,83.0,NPS,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,R05.2015.986001


In [11]:
df_AR.sample(5)

,WBS1,WBS2,WBS3,Invoice,Period,InvoiceDate,PaidPeriod,InvBalanceSourceCurrency
6564,R01.2007.005423,3,1,15955,200910,2009-10-22,200911,0.0
33814,R08.2007.000062,1,2,14105,200801,2008-01-31,200803,0.0
17566,R05.2007.000329,13,2,20626,201403,2014-03-06,201404,0.0
29075,R01.2003.003630,1,MOBILIZ,10837,200412,2004-12-31,200412,0.0
10631,R05.2015.986001,9,1,25435,202010,2020-10-31,202011,0.0


### Peroid vs. PaidPeriod dates, approximate delay in payment as new column.
Try to use InvoiceDate to further estimate?
May need to bring in column from some other table to zero in on actual paid date.

In [10]:
df_AR.columns

Index(['WBS1', 'WBS2', 'WBS3', 'Invoice', 'Period', 'InvoiceDate',
       'PaidPeriod', 'InvBalanceSourceCurrency'],
      dtype='object')

In [11]:
df_AR['InvoiceDate'][0]

Timestamp('2007-03-16 00:00:00')

In [12]:
df_AR['PaidPeriod'][0]

200705

In [29]:
df_AR['Period']

0        200703
1        200706
2        200711
3        200709
4        200712
          ...  
36911    201204
36912    201206
36913    201302
36914    201305
36915    201305
Name: Period, Length: 36916, dtype: int64

In [37]:
# Convert 'PaidPeriod' and 'Period' to datetime objects
df_AR['PaidPeriod'] = pd.to_datetime(df_AR['PaidPeriod'], format='%Y%m')
df_AR['Period'] = pd.to_datetime(df_AR['Period'], format='%Y%m')

# Calculate the difference in months
df_AR['DateDifference'] = (df_AR['PaidPeriod'] - df_AR['Period']).dt.days

ValueError: unconverted data remains: 9

In [11]:
test_filepath = [filepath for filepath in file_path_list][2]

try:
    pd.read_csv(test_filepath)
except:
    with open(test_filepath, 'rb') as f:
        result = chardet.detect(f.read())  # or readline if the file is large
    
    dd.read_csv(test_filepath, encoding=result['encoding']).compute()

NameError: name 'chardet' is not defined

In [ ]:
# Save the list of filenames and full paths
with open('file_list.txt', 'w') as f:
    for file_name, full_path in zip(file_names, full_paths):
        f.write(f'{file_name}\t{full_path}\n')

In [ ]:
try:
    dd.read_csv(test_filepath)

try:
    pd.read_csv(test_filepath)
except:
    with open(test_filepath, 'rb') as f:
        result = chardet.detect(f.read())  # or readline if the file is large
    
    dd.read_csv('Vision/CL.csv', encoding=result['encoding']).compute()
    
except ValueError as ve:
    error_message = str(ve)
    print(f"Error message: {error_message}")
#     problematic_columns = set(re.findall(r"'([^']+)'", error_message))
#     print(f"Problematic columns: {problematic_columns}")

In [ ]:
dd.read_csv(test_filepath)

In [ ]:
df_dict = {"names": [], "filepaths": [], "dataframes": []};
for i, file_path in enumerate(file_path_list, 1):
    df_name = "df_" + str(i)
    path = file_path
    df_dict["names"].append(df_name)
    df_dict["filepaths"].append(path)
    
    try:
        df = dd.read_csv(file_path)
        df_dict["dataframes"].append(df)
    except:
        with open 
       
    

df_dict

In [ ]:
len(df_dict['dataframes'])

In [ ]:
def csv_batch_import(csv_file_paths):
    for i, file_path in enumerate(csv_file_paths, 1):
        try:
            df = dd.read_csv(file_path)
            globals()[f'df_{i}'] = df.compute()
            print(f'Successfully loaded {file_path} as df_{i}')
        except Exception as e:
            print(f'Error occurred with {file_path}: {str(e)}')

# Call the function
csv_batch_import(file_path_list)